In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random as rand

class parameter:
    def __init__(self):
        self.t = 0.0
        self.dt = 1e-4
        self.t_counter = [0., 1., 10., 25.]
        self.tlim = 25
        self.x = 0.0
        self.dx = 2e-2
        self.xlim = 25
        self.y = 0.0
        self.dy = 2e-2
        self.ylim = 25
        self.Re = 100#500, 1000
        self.Er = 1.0e-3
        self.Etr = 1.0e-8
        self.air_rho = 0.00129
        self.sand_rho = 2.65
        self.U0 = 1.0


class air:
    def __init__(self, p: parameter):
        self.u = [[0 for i in range((int(1./p.dx)))] for j in range(int(1./p.dy))]
        self.v = [[0 for i in range((int(1./p.dx)))] for j in range(int(1./p.dy))]
        self.p = [[0 for i in range((int(1./p.dx)))] for j in range(int(1./p.dy))]
        self.p_star = self.p.copy()
        self.u_star = self.u.copy()
        self.v_star = self.v.copy()
        self.A = 2.0 * (1./p.dx**2 + 1./p.dy**2)
        self.B = 0.0
        self.P = 0.0
        self.alpha = -0.5#適当なのであしからず
    
    def setup(self, p:parameter)->None:
        #****壁側(両端)の条件*******#
        for i in range(len(self.u)):
            self.u[i][0] = self.u[i][0] = 0.
        for i in range(len(self.v)):
            self.v[i][0] = self.v[i][0] = 0.
        #****lid(一番上のところ)の条件****#
        for i in range(len(self.u[len(self.u)-1])):
            self.u[len(self.u)-1][i] = p.U0
        for i in range(len(self.v[len(self.v)-1])):
            self.v[len(self.v)-1][i] = 0.
        
        
    def calc_ustar(self, p:parameter, i:int, j:int)->None:
         self.u_star[j][i] = self.u[j][i] - p.dt*(self.u[j][i] * (self.u[j][i+1] - self.u[j][i-1])/(2*p.dx) +\
                                                 (self.v[j-1][i] + self.v[j][i] + self.v[j-1][i+1] + self.v[j][i+1])/4.0 * \
                                                 (self.u[j+1][i] - self.u[j-1][i]) /(2*p.dy) -\
                                                 1./p.Re * ((self.u[j][i+1] - 2 * self.u[j][i] + self.u[j][i-1]) / p.dx**2 + \
                                                (self.u[j+1][i] - 2*self.u[j][i] + self.u[j-1][i])/p.dy**2))
            
    def calc_vstar(self, p:parameter, i:int, j:int)->None:
        self.v_star[j][i] = self.v[j][i] - p.dt*(self.v[j][i] * (self.v[j+1][i] - self.v[j-1][i])/(2*p.dy) +\
                                                 (self.u[j-1][i] + self.u[j][i] + self.u[j-1][i+1] + self.u[j][i+1])/4.0 * \
                                                 (self.v[j][i+1] - self.v[j][i-1])/(2*p.dx) -\
                                                 1./p.Re * ((self.v[j][i+1] - 2 * self.v[j][i] + self.u[j][i-1])/p.dx**2 +\
                                                           (self.v[j+1][i] -2*self.v[j][i] + self.v[j-1][i])/p.dy**2))
    
    def calc_pstar(self, p:parameter, i:int, j:int)->None:
        self.B      = p.air_rho / p.dt * ((self.u_star[j][i+1] - self.u_star[j][i-1])/p.dx +\
                                                     (self.v_star[j+1][i] - self.v_star[j-1][i]) /p.dy)
        self.P      = (self.p[j][i-1] + self.p[j][i+1]) / p.dx**2 + (self.p[j-1][i] + self.p[j+1][i])/p.dy**2
        self.p_star[j][i] = (1 + self.alpha)*(self.P - self.B) / self.A - self.alpha*self.p[j][i]
    
    def calc_u(self, p:parameter, i:int, j:int)->None:
        self.u[j][i] = self.u_star[j][i] - p.dt / p.air_rho *(self.p_star[j][i+1] - self.p_star[j][i-1])/p.dx
        
    def calc_v(self, p:parameter, i:int, j:int)->None:
        self.v[j][i] = self.v_star[j][i] - p.dt / p.air_rho *(self.p_star[j+1][i] - self.p_star[j-1][i])/p.dy
        
    def revival(self, p:parameter, i:int, j:int)->None:
        tmp_u = self.u[j][i]
        tmp_v = self.v[j][i]
        self.calc_ustar(p, i, j); self.calc_vstar(p, i, j)##Solve u_star and v_star
        self.calc_pstar(p, i, j)                          ##Solve p_star using u_star and v_star
        self.calc_u(p, i, j); self.calc_v(p, i, j)        ##Correct u' and v' using u_star, v_star and p_star
        #if(p.Er < abs(tmp_u - self.u[j][i]) and p.Er < abs(tmp_v - self.v[j][i])):
         #   self.revival(p, i, j)
    
    def MAC(self, p:parameter)->None:
        p.t += p.dt
        for j in range(1, len(self.u) - 1):
            for i in range(1, len(self.u) - 1):
                tmp_u = self.u[j][i]
                tmp_v = self.v[j][i]
                self.calc_ustar(p, i, j); self.calc_vstar(p, i, j)##Solve u_star and v_star
                self.calc_pstar(p, i, j)                          ##Solve p_star using u_star and v_star
                self.calc_u(p, i, j); self.calc_v(p, i, j)        ##Correct u' and v' using u_star, v_star and p_star
                while(p.Er < np.fabs(tmp_u - self.u[j][i]) and p.Er < np.fabs(tmp_v - self.v[j][i])):
                    #print('i = {0:2d}, j = {1:2d}, error of u is {2:10e}, error of v is {3:10e}'.format(i, j, np.fabs(tmp_u-self.u[j][i]), np.fabs(tmp_v-self.v[j][i])))
                    self.revival(p, i, j)
        print()
        # ディリクレ条件
        for i in range(len(self.u[0])):
            self.u_star[0][i] = 0.#-self.u_star[1][i]
            self.u[i][0] = 0.#-self.u[i][1]
            self.u_star[i][len(self.u[i])-1] = 0.#-self.u_star[i][len(self.u[i])-2]
            self.u[i][len(self.u[i])-1] = 0.#-self.u[i][len(self.u[i])-2]
        for i in range(len(self.v[0])):
            self.v_star[0][i] = 0.#-self.v_star[1][i]
            self.v[0][i] = 0.#-self.v[1][i]
            self.v_star[i][len(self.v[i])-1] = 0.#-self.v_star[i][len(self.v[i])-2]
            self.v[i][len(self.v[i])-1] = 0.#-self.v[i][len(self.v[i])-2]
        print(self.u)
        print(self.v)
        
if __name__ == '__main__':
    p = parameter()
    a = air(p)
    a.setup(p)
    while np.fabs(p.t - p.tlim) > p.Etr:
        print('\r t = {0:2.5f}'.format(p.t), end=' ')
        a.MAC(p)